# Equal-Weight Top 100 Cryptocurrencies Investment

#Introduction

In this exercise we will create a python script that will accept the amount we are willing to invest i.e, our portfolio size and tell us how many tokens of each cryptocurrency from the Top 100 cryptocurrencies(by market capitalization) we should purchase to get an equal-weight distribution of our investment amount.

#Library Imports

First, we will import the list of open source software libraries that we'll be using in this exercise.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math

#Importing List of Top 100 Cryptocurrencies

The next thing we need to do is import the list of top 100 cryptocurrencies as on https://www.coingecko.com/.

The cryptocurrencies in this list may change over time so the ideal way to get the list of top 100 cryptocurrencies is to connect to the Coingecko API and get the real time data from their API database. The API documentation can be found here: https://www.coingecko.com/en/api/documentation.

Since this is a one time exercise, we will be using the free API version, there is no need to get an API key now. However, if we need to access the database frequently and choose a paid plan, an API key will be required. We will be using the /coins/markets API end point to get the list of top 100 cryptocurrencies. The end point includes other relevant data as well, such as, ticker, price, market capitalization, etc which we will need later on.

Let's import the list to our Jupyter notebook using the following commands:

In [2]:
api_url = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=1&sparkline=false'
data = requests.get(api_url).json()

#Creating a Pandas DataFrame and parsing our API call

In order to create an equal-weighted script, we will use the following information from the API:

Name
Ticker
Current price
Market capitalization
We will parse the data that we obtained above from the API request to extract the abovementioned values. We will store the obtained information in a dataframe and also add a fifth column to our dataframe that outputs the number of tokens to buy for each cryptocurrency.

In [4]:
my_columns = ['Name', 'Ticker', 'Current Price', 'Market Capitalization', 'Number of Tokens to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Name,Ticker,Current Price,Market Capitalization,Number of Tokens to Buy


In [5]:
for i in data:
    name = i['name']
    ticker = i['symbol']
    price = i['current_price']
    market_cap = i['market_cap']
    final_dataframe = final_dataframe.append(
    pd.Series(
        [
            name,
            ticker,
            price,
            market_cap,
            'N/A'
        ],
            index = my_columns,
    ),
        ignore_index = True
)
final_dataframe

,Name,Ticker,Current Price,Market Capitalization,Number of Tokens to Buy
0,Bitcoin,btc,43297,821325950916,N/A
1,Ethereum,eth,2897.61,347026358638,N/A
2,Tether,usdt,1.001,79610554781,N/A
3,BNB,bnb,406.56,68362296179,N/A
4,USD Coin,usdc,1.001,53602313382,N/A
...,...,...,...,...,...
95,Loopring,lrc,0.808862,1007735560,N/A
96,Convex Finance,cvx,18.6,972286138,N/A
97,Curve DAO Token,crv,2.46,964550711,N/A
98,NEM,xem,0.105726,951432623,N/A


#Calculating the Number of Tokens to Buy

Next, we will calculate the number of tokens to buy for each cryptocurrency.

In [6]:
portfolio_size = input("Enter the value of your portfolio:")
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [7]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
   final_dataframe.loc[i, 'Number of Tokens to Buy'] = math.floor(position_size / final_dataframe['Current Price'][i])
final_dataframe

,Name,Ticker,Current Price,Market Capitalization,Number of Tokens to Buy
0,Bitcoin,btc,43297,821325950916,0
1,Ethereum,eth,2897.61,347026358638,3
2,Tether,usdt,1.001,79610554781,9990
3,BNB,bnb,406.56,68362296179,24
4,USD Coin,usdc,1.001,53602313382,9990
...,...,...,...,...,...
95,Loopring,lrc,0.808862,1007735560,12363
96,Convex Finance,cvx,18.6,972286138,537
97,Curve DAO Token,crv,2.46,964550711,4065
98,NEM,xem,0.105726,951432623,94584


Finally, we will output the above dataframe result in an excel file using xlsxwriter.

In [8]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [9]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [10]:
column_formats = { 
                    'A': ['Name', string_format],
                    'B': ['Ticker', string_format],
                    'C': ['Current Price', dollar_format],
                    'D': ['Market Capitalization', dollar_format],
                    'E': ['Number of Tokens to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [11]:
writer.save()